<a href="https://colab.research.google.com/github/dashatenoff/recsys-vk/blob/main/notebooks/ALS_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implicit ALS Recommender System (VK-LSVD)

В этом ноутбуке реализована модель рекомендательной системы
на основе **implicit feedback** с использованием алгоритма
**Alternating Least Squares (ALS)**.

В качестве данных используется подмножество датасета VK-LSVD,
содержащее взаимодействия пользователей с видеоклипами.

Цель работы — построить baseline-модель рекомендаций,
оценить её качество по метрике **MAP@10**
и проанализировать влияние гиперпараметров ALS.


In [ ]:
from tqdm import tqdm
tqdm.disable = True


import pandas as pd
from google.colab import drive
import os
from scipy.sparse import csr_matrix
from collections import defaultdict

drive.mount('/content/drive')
os.listdir('/content/drive/MyDrive')
train = pd.read_parquet('/content/drive/MyDrive/VK/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/VK/test.parquet')

In [ ]:
train.head(10)
train['timespent'].describe()

Анализ распределения `timespent` показал отсутствие коротких просмотров
(минимальное значение — 6 секунд), поэтому фильтрация по времени просмотра
не применяется. Используется бинарный implicit-сигнал.


#Маппинг
## Подготовка данных

ALS работает с матрицей взаимодействий пользователей и объектов.
Поскольку исходные `user_id` и `item_id` имеют большие значения,
используется маппинг в компактные индексы.

В качестве сигнала используется **бинарный implicit feedback**:
факт взаимодействия пользователя с видео.


In [ ]:
user_to_index = {}
item_to_index = {}
rows, cols, data = [], [], []

for row in train.itertuples():
  u = user_to_index.setdefault(row.user_id, len(user_to_index))
  i = item_to_index.setdefault(row.item_id, len(item_to_index))

  rows.append(u)
  cols.append(i)
  data.append(1)

user_item_matrix = csr_matrix((data, (rows, cols)))


## Обучение ALS модели

Используется реализация ALS из библиотеки `implicit`,
оптимизированная для работы с разреженными матрицами
и неявной обратной связью.

Модель обучается на всей матрице взаимодействий train-выборки.


In [ ]:
!pip install implicit


In [ ]:
import implicit



als_model = implicit.als.AlternatingLeastSquares(
    factors=16,
    regularization=10,
    alpha=1000,
    iterations=10
)

als_model.fit(user_item_matrix)


## Генерация рекомендаций

Для каждого пользователя генерируется топ-10 рекомендаций.
Из рекомендаций исключаются объекты,
с которыми пользователь уже взаимодействовал.



In [ ]:
index_to_item = {v: k for k, v in item_to_index.items()}

def recommend_for_user_als(user_id, k=10):
  if user_id not in user_to_index:
    return []

  user_idx = user_to_index[user_id]

  item_indices, scores = als_model.recommend(
      user_idx,
      user_item_matrix[user_idx],
      N = k,
      filter_already_liked_items=True
  )
  return [index_to_item[i] for i in item_indices]

test_user = train['user_id'][0]
recommend_for_user_als(test_user, k=10)

#Генерируем рекомендации для ВСЕХ пользователей test

In [ ]:
recs = []
test_user = test['user_id'].unique()

for user_id in test_user:
  recs_items = recommend_for_user_als(user_id, k=10)
  for item in recs_items:
    recs.append({
        'user_id' : user_id,
        'recs' : item
    })
submission = pd.DataFrame(recs)
submission.to_csv('submission_als', index=False)
submission.head(20)


#MAP@10
## Оценка качества

Качество рекомендаций оценивается по метрике **MAP@10**,
которая учитывает порядок рекомендаций и усредняется по пользователям.


In [ ]:
from string import octdigits
rel = test.groupby('user_id')['item_id'].apply(set)
pred = submission.groupby('user_id')['recs'].apply(list)

aps = []
for user_id in pred.index:
  ord = 1
  cor = 0
  score =0.0
  for item_id in pred[user_id]:
    if item_id in rel.get(user_id, set()):
      cor+=1
      score+=cor/ord
    ord+=1
  aps.append(score/min((len(rel.get(user_id, [])), 10)))
map10 = sum(aps)/len(aps)
map10

#Сравниваем конфигурации


Были протестированы две конфигурации ALS,
отличающиеся параметром `alpha`,
который определяет степень доверия к implicit-сигналу.

Цель — проверить влияние этого параметра на качество рекомендаций.


In [ ]:
index_to_item = {v: k for k, v in item_to_index.items()}
rel = test.groupby('user_id')['item_id'].apply(set)

results = []

for cfg in configs:
    print(f"\nTraining ALS with config: {cfg}")

    # 1. Обучаем модель
    als_model = implicit.als.AlternatingLeastSquares(
        factors=cfg['factors'],
        regularization=cfg['reg'],
        alpha=cfg['alpha'],
        iterations=10
    )
    als_model.fit(user_item_matrix)

    # 2. Генерируем рекомендации
    recs = []
    for user_id in test['user_id'].unique():
        if user_id not in user_to_index:
            continue

        user_idx = user_to_index[user_id]

        item_indices, _ = als_model.recommend(
            user_idx,
            user_item_matrix[user_idx],
            N=10,
            filter_already_liked_items=True
        )

        for item_idx in item_indices:
            recs.append({
                'user_id': user_id,
                'recs': index_to_item[item_idx]
            })

    submission = pd.DataFrame(recs)

    # 3. Считаем MAP@10
    pred = submission.groupby('user_id')['recs'].apply(list)

    aps = []
    for user_id in pred.index:
        ord = 1
        cor = 0
        score = 0.0
        for item_id in pred[user_id]:
            if item_id in rel.get(user_id, set()):
                cor += 1
                score += cor / ord
            ord += 1
        aps.append(score / min(len(rel.get(user_id, [])), 10))

    map10 = sum(aps) / len(aps)

    print(f"MAP@10 = {map10:.5f}")

    # 4. Сохраняем результат
    results.append({
        'factors': cfg['factors'],
        'alpha': cfg['alpha'],
        'regularization': cfg['reg'],
        'map10': map10
    })


In [ ]:
results_df = pd.DataFrame(results)
results_df


Более агрессивная конфигурация (`alpha = 1000`)
показала лучшее качество по MAP@10,
что соответствует ожиданиям для плотных implicit-логов.


## Результаты

Качество модели ALS оценивалось с помощью метрики **MAP@10**.

Были протестированы две конфигурации модели:

- `alpha = 40`, `regularization = 0.1` → **MAP@10 = 0.0129**
- `alpha = 1000`, `regularization = 10` → **MAP@10 = 0.0164**

Увеличение параметра `alpha`, отвечающего за степень доверия
к неявной обратной связи (implicit feedback),
привело к улучшению качества рекомендаций на данном датасете.
